In [18]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import glob,pandas,time

import torch,torch.nn,torchvision
from torch.utils.data import Dataset,DataLoader
from torchsummary import summary

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Mean&STD

In [19]:
train_fp = "../01DataPreProcessing/crop_img/"
RGB_mean = torch.tensor([0,0,0],dtype=torch.float32)
RGB_std = torch.tensor([0,0,0],dtype=torch.float32)
transform = torchvision.transforms.ToTensor()
n = 0
for fp in glob.glob("../01DataPreProcessing/crop_img/*.jpg"):
    with Image.open(fp) as img:
        img = transform(img)
        RGB_mean += img.mean(dim=(1,2))
        RGB_std += img.std(dim=(1,2))
        n+=1
RGB_mean /= n
RGB_std /= n

Dataset

In [79]:
class MyDataSet(Dataset):
    def __init__(self,file_paths,labels_lists,transform):
        super().__init__()
        self.file_paths = file_paths
        self.labels_lists = labels_lists
        self.transform = transform
    def __len__(self):
        return len(self.file_paths)
    def __getitem__(self, index):
        if torch.is_tensor(index):
            index = index.tolist()
        image = Image.open(self.file_paths[index])
        image = self.transform(image).float()
        labels = self.labels_lists[index]
        labels = np.array(labels)
        labels = labels.astype('float').reshape(-1,4)
        sample = {'image':image,'psi':labels}
        return sample

In [32]:
# images_fp,labels_fp = "../01DataPreProcessing/crop_img/*.jpg","../01DataPreProcessing/ftt_psi.csv"
# df = pandas.read_csv(labels_fp)
# df = df.iloc[:,20:24]
# psi = df.to_numpy()
# img_pl = glob.glob(images_fp)
# img_pl = np.array(img_pl)

# #shuffle
# orig_dataset = np.c_[img_pl,psi]
# np.random.seed(1234)
# np.random.shuffle(orig_dataset)

DataLoader

In [97]:
def BuildDataLoader(images_fp,labels_fp,input_size):
    df = pandas.read_csv(labels_fp)
    df = df.iloc[:,20:24]
    psi = df.to_numpy()
    img_pl = glob.glob(images_fp)
    img_pl = np.array(img_pl)

    #shuffle
    orig_dataset = np.c_[img_pl,psi]
    np.random.seed(1234)
    np.random.shuffle(orig_dataset)

    #shuffle
    orig_dataset = np.c_[img_pl,psi]
    np.random.seed(1234)
    np.random.shuffle(orig_dataset)

    #split into train and test
    train_inputs, test_inputs = [],[]
    train_labels, test_labels = [],[]
    train_num = int(len(orig_dataset)*0.8)
    for data in orig_dataset[:train_num]:
        train_inputs.append(data[0])
        train_labels.append([data[1:]])
    for data in orig_dataset[train_num:]:
        test_inputs.append(data[0])
        test_labels.append([data[1:]])

    # normalize = torchvision.transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
    normalize = torchvision.transforms.Normalize(mean = RGB_mean.tolist(), std = RGB_std.tolist())
    data_transforms = {
        'train': torchvision.transforms.Compose([
            torchvision.transforms.RandomResizedCrop(input_size),
            torchvision.transforms.RandomHorizontalFlip(),
            torchvision.transforms.ToTensor(),
            normalize
        ]),
        'val': torchvision.transforms.Compose([
            torchvision.transforms.Resize(input_size),
            torchvision.transforms.CenterCrop(input_size),
            torchvision.transforms.ToTensor(),
            normalize
        ]),
        'test': torchvision.transforms.Compose([
            torchvision.transforms.Resize(input_size),
            torchvision.transforms.CenterCrop(input_size),
            torchvision.transforms.ToTensor(),
            normalize
        ])
    }
    # train_transformer = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),normalize])
    # test_transformer = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),normalize])

    train_dataLoader = DataLoader(MyDataSet(train_inputs,train_labels,data_transforms['train']),batch_size=3,shuffle=True)
    test_dataLoader = DataLoader(MyDataSet(test_inputs,test_labels,data_transforms['test']),batch_size=3,shuffle=False)
    return train_dataLoader,test_dataLoader


In [98]:
train_dataLoader,test_dataLoader = BuildDataLoader("../01DataPreProcessing/crop_img/*.jpg","../01DataPreProcessing/ftt_psi.csv",244)

ResNet18

In [ ]:
model_resnet50 = torchvision.models.resnet50(weights=torchvision.models.ResNet50_Weights.DEFAULT)
print(model_resnet50)

In [111]:
pre_model_Resnet = torchvision.models.resnet18(weights=torchvision.models.ResNet18_Weights.DEFAULT)
# print(pre_model_Resnet)
#凍結參數
for param in pre_model_Resnet.parameters():
    param.requires_grad = False

# pre_model_Resnet.fc = torch.nn.Linear(in_features=512,out_features=4)
pre_model_Resnet.fc = torch.nn.Sequential(torch.nn.Linear(in_features=512,out_features=128),torch.nn.Sigmoid(),torch.nn.Linear(in_features=128,out_features=4))
pre_model_Resnet = pre_model_Resnet.to(device=device)
print(pre_model_Resnet)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [112]:
params_to_update = []
for name,param in pre_model_Resnet.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)

	 fc.0.weight
	 fc.0.bias
	 fc.2.weight
	 fc.2.bias


In [113]:
optimizer = torch.optim.SGD(params_to_update, lr=0.001)
criterion = torch.nn.MSELoss()
summary(pre_model_Resnet,(3,244,244))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 122, 122]           9,408
       BatchNorm2d-2         [-1, 64, 122, 122]             128
              ReLU-3         [-1, 64, 122, 122]               0
         MaxPool2d-4           [-1, 64, 61, 61]               0
            Conv2d-5           [-1, 64, 61, 61]          36,864
       BatchNorm2d-6           [-1, 64, 61, 61]             128
              ReLU-7           [-1, 64, 61, 61]               0
            Conv2d-8           [-1, 64, 61, 61]          36,864
       BatchNorm2d-9           [-1, 64, 61, 61]             128
             ReLU-10           [-1, 64, 61, 61]               0
       BasicBlock-11           [-1, 64, 61, 61]               0
           Conv2d-12           [-1, 64, 61, 61]          36,864
      BatchNorm2d-13           [-1, 64, 61, 61]             128
             ReLU-14           [-1, 64,

In [114]:
loss_epoch_C = []
train_acc, test_acc = [], []
best_acc, best_auc = 0.0, 0.0
epochs = 50

In [115]:
for epoch in range(epochs):
        start_time = time.time()
        iter = 0
        correct_train, total_train = 0, 0
        correct_test, total_test = 0, 0
        train_loss_C = 0.0
        total_mae = 0.0

        pre_model_Resnet.train() # 設定 train 或 eval
        print('epoch: ' + str(epoch + 1) + ' / ' + str(epochs))  
        
        # ---------------------------
        # Training Stage
        # ---------------------------
        for i, d in enumerate(train_dataLoader) :
            x, label = d['image'].to(device), d['psi'].to(device).float().squeeze(dim=1)
            # print(label.shape)
            optimizer.zero_grad()                           # 清空梯度
            train_output = pre_model_Resnet(x)              # 將訓練資料輸入至模型進行訓練 (Forward propagation)
            # print(label.size)
            train_loss = criterion(train_output, label)     # 計算 loss
            train_loss.backward()                           # 將 loss 反向傳播
            optimizer.step()                                # 更新權重

            train_loss_C += train_loss.item()
            mae = (torch.abs(train_output - label).sum()/4).item()
            total_mae += mae    
            
            # 計算訓練資料的準確度 (correct_train / total_train)
            # _, predicted = torch.max(train_output.data, 1)  # 取出預測的 maximum
            # total_train += label.size(0)
            # correct_train += (predicted == label).sum()
            # train_loss_C += train_loss.item()
            iter += 1
                    
        print('Training epoch: %d / loss_C: %.3f | acc: %.3f' % (epoch + 1, train_loss_C / iter, 1-total_mae / iter))
        
        # --------------------------
        # Testing Stage
        # --------------------------
        pre_model_Resnet.eval() # 設定 train 或 eval
        iter = 0
        total_mae = 0.0
        for i, d in enumerate(test_dataLoader) :
            with torch.no_grad():                           # 測試階段不需要求梯度
                x, label = d['image'].to(device), d['psi'].to(device).float().squeeze(dim=1)
                test_output = pre_model_Resnet(x)                          # 將測試資料輸入至模型進行測試
                test_loss = criterion(test_output, label)   # 計算 loss
                
                mae = (torch.abs(test_output - label).sum()/4).item()
                total_mae += mae
                iter+=1
                # 計算測試資料的準確度 (correct_test / total_test)
                # _, predicted = torch.max(test_output.data, 1)
                # total_test += label.size(0)
                # correct_test += (predicted == label).sum()
        
        print('Testing acc: %.3f' % (1-total_mae / iter))
                                     
        # train_acc.append(100 * (correct_train / total_train).cpu()) # training accuracy
        # test_acc.append(100 * (correct_test / total_test).cpu())    # testing accuracy
        # loss_epoch_C.append((train_loss_C / iter))            # loss 

        end_time = time.time()
        print('Cost %.3f(secs)' % (end_time - start_time))

epoch: 1 / 50
Training epoch: 1 / loss_C: 0.212 | acc: -0.034
Testing acc: 0.295
Cost 18.963(secs)
epoch: 2 / 50
Training epoch: 2 / loss_C: 0.160 | acc: 0.206
Testing acc: 0.314
Cost 19.594(secs)
epoch: 3 / 50
Training epoch: 3 / loss_C: 0.157 | acc: 0.214
Testing acc: 0.274
Cost 20.264(secs)
epoch: 4 / 50
Training epoch: 4 / loss_C: 0.155 | acc: 0.215
Testing acc: 0.302
Cost 20.222(secs)
epoch: 5 / 50
Training epoch: 5 / loss_C: 0.155 | acc: 0.219
Testing acc: 0.279
Cost 20.552(secs)
epoch: 6 / 50
Training epoch: 6 / loss_C: 0.158 | acc: 0.214
Testing acc: 0.257
Cost 20.477(secs)
epoch: 7 / 50
Training epoch: 7 / loss_C: 0.155 | acc: 0.220
Testing acc: 0.297
Cost 20.437(secs)
epoch: 8 / 50
Training epoch: 8 / loss_C: 0.153 | acc: 0.221
Testing acc: 0.281
Cost 20.637(secs)
epoch: 9 / 50
Training epoch: 9 / loss_C: 0.153 | acc: 0.225
Testing acc: 0.293
Cost 20.953(secs)
epoch: 10 / 50
Training epoch: 10 / loss_C: 0.153 | acc: 0.224
Testing acc: 0.314
Cost 20.908(secs)
epoch: 11 / 50
Tr